<a href="https://colab.research.google.com/github/munkefrugt/StockPrograms/blob/main/BACK%20UP_WORKING%20Backtester_winning_%20MACD_trading_rush_23%20Feb%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 KB 9.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [2]:
import yfinance as yf
import plotly.graph_objects as go

import numpy as np

import pandas as pd
from plotly.subplots import make_subplots
import math

#import mplfinance as mpf

In [55]:



# start_date = '2020-02-01'
# end_date = '2020-03-01'

# df = yf.download(tickers="SPY", start=start_date, end=end_date, interval='30m')
df = yf.download(tickers="SPY", period="59d", interval='30m')



[*********************100%***********************]  1 of 1 completed


In [56]:
df['row_number'] = df.reset_index(drop=False).index + 1


df.insert(0, "Row_Number", range(1, len(df) + 1))
df = df.drop(df.columns[-1], axis=1)
df

,Row_Number,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,,
2022-11-28 09:30:00,1,399.089996,400.809998,399.000000,399.867096,399.867096,6205462
2022-11-28 10:00:00,2,399.850006,400.239990,398.825195,399.350006,399.350006,4409803
2022-11-28 10:30:00,3,399.299988,399.549988,398.410004,398.510010,398.510010,3606058
2022-11-28 11:00:00,4,398.510010,398.869995,398.190002,398.540009,398.540009,2539879
2022-11-28 11:30:00,5,398.540009,399.079987,398.350098,398.872009,398.872009,2779621
...,...,...,...,...,...,...,...
2023-02-22 13:30:00,760,400.785004,400.880005,400.000000,400.359894,400.359894,2490140
2023-02-22 14:00:00,761,400.356293,401.130005,398.600006,399.190002,399.190002,9431314
2023-02-22 14:30:00,762,399.190002,399.289703,397.510010,397.589905,397.589905,7549266


In [45]:
df['EMA50'] = df['Close'].ewm(span=50, adjust=False).mean()
df['EMA100'] = df['Close'].ewm(span=100, adjust=False).mean()
df['EMA200'] = df['Close'].ewm(span=200, adjust=False).mean()
df['EMA500'] = df['Close'].ewm(span=500, adjust=False).mean()
df['EMA2000'] = df['Close'].ewm(span=2000, adjust=False).mean()

# Calculate MACD
df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA12'] - df['EMA26']
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

# 
window = 20
#Donchian channel
df['Upper_band'] = df['High'].rolling(window=window).max()

df['Stop_Loss'] = df['Low'].rolling(window=window).min()

In [46]:
#make signals:



#buy sugnals: 
df['Long_Entry'] = np.where(
  #locate latest max
  (df['MACD'] < 0) & 
  (df['Signal'] < 0) & 
  (df['MACD'].shift(1) < df['Signal'].shift(1)) & 
  (df['MACD'] > df['Signal']) &
  #close above ema200
  
  (df['Close'] > df['EMA200']) &
  (df['Row_Number'] > window),
   1, 0) 

buy_signals = df[df['Long_Entry'] == 1]
num_long_entries = df['Long_Entry'].sum()
print("Number of long entries:", num_long_entries)

#sell signal:
# Calculate the stop loss line

# Calculate exit signals

#df['Long_Exit'] = np.where( ( df['Close']) < df['Stop_Loss'].shift(1), 1, 0) 
df['Long_Exit'] = 0
# Get the sell signals
sell_signals = df[df['Long_Exit'] == 1]
num_long_exits = df['Long_Exit'].sum()
print("Number of long exits:", num_long_exits)

Number of long entries: 5
Number of long exits: 0


In [47]:

class Trade:
    def __init__(self, entry_price, cash, risk_percentage, df=None, i=None, trade_id = None):
        self.entry_price = entry_price
        self.cash = cash
        #self.equity = equity
        self.risk_percentage = risk_percentage
        self.df = df
        self.i = i # current index
        self.tp2 = None
        self.sl = self.df.loc[self.i, "Stop_Loss"]
        self.exit_price = None
        self.status = "open"
        self.risk = entry_price - self.sl
        self.tp1 = self.entry_price + self.risk * 1.5

        self.profit = None
        
        self.pot_gain = None
        self.pot_loss = None
        self.trade_cost = None
        self.position_size = None
        self.trade_id = trade_id

        print("self.trade_id : " + str(self.trade_id))
        #print("self.entry_price" + str(self.entry_price))

    
    def set_tp(self):
        self.tp1 = self.entry_price + self.risk * 1.5
        print("self.tp1 " +str(self.tp1))

    def set_sl(self):
        self.sl = self.df.loc[self.i, "Stop_Loss"]
        print("self.sl " +str(self.sl))

    def get_potential_loss(self):
        self.pot_loss = -self.position_size * self.risk
                             
    def get_potential_gain(self):
        self.pot_gain = self.position_size * self.risk*1.5
        print("self.pot_gain : "+ str(self.pot_gain))

    def get_trade_cost(self):
        self.trade_cost = self.position_size * self.entry_price
      
    def get_profit(self, current_price):  
        inital_investment = self.position_size* self.entry_price
        current_investment = self.position_size* current_price
        self.profit = current_investment- inital_investment
        return self.profit

In [48]:


#backtest
print("backtest\n\n")
#position_size=0.01
start_capital = 10000
equity=start_capital
cash = start_capital
risk_percentage = 0.005
trades = []
winning_trades = 0
losing_trades = 0
current_winning_streak = 0
current_losing_streak = 0
max_winning_streak = 0
max_losing_streak = 0
total_position_amount = 0

id_counter = 0
df_trades = pd.DataFrame(columns=["trade_id"])
df_trades.set_index("trade_id", inplace=True)

df["Potential_loss_over_equity_pct"] =0

for i, row in df.iterrows():

    current_price = row["Open"]
    signal = df.loc[i, "Long_Entry"]
    
        

    #update cash
    df.loc[i, "Cash"] = cash
    equity = (total_position_amount * current_price) + cash
    df.loc[i, "Equity"] = equity
    #update total_position_amount
    df.loc[i, "Total_position_amount"] = total_position_amount



    # Check if a new trade needs to be opened

    if signal == 1:
        print("==================================>buy signal <==================================")
        print("\n\n")
        print("equity : "+str(equity))
        print("cash : " + str(cash))
        print("total_position_amount before potential trade : " +str(total_position_amount))
        print("current_price: " +str(current_price))

        print("index = " + str(i))

        # check if y cash availible to trade for: 
        if cash >= current_price : 
          # calculate position size:
          print("calculate_position_size")

          risk = current_price- df.loc[i, "Stop_Loss"]
          max_position_size = (equity * risk_percentage) / risk
          #print("max_position_size" + str(max_position_size))


          position_size = math.floor(max_position_size)
          print("max position size "  + str(position_size))
          potential_spending = position_size * current_price

          max_loss = equity * risk_percentage
          potential_loss = position_size* risk
          if potential_spending > cash:
            # error its not the right position size. It need to be smaller. 
            position_size = math.floor(cash / current_price)
            potential_spending = position_size *current_price
            print("change position amount to " + str(position_size))
          else: 
            
            print("use max position size "  + str(position_size))


          if potential_loss <= max_loss :
            print("we can trade")
            print("potential_loss " + str(potential_loss))
            potential_loss_over_equity = potential_loss/ equity
            df.loc[i, "Potential_loss_over_equity_pct"] = potential_loss_over_equity * 100

            print("potential_spending " + str(potential_spending))
            print("buy position size "  + str(position_size))

            cash =cash- potential_spending
            print("cash left " +str(cash))

            

            # Open a new trade
            print("Open Trade ! <-------------")
            trade = Trade(entry_price=current_price, cash=cash, risk_percentage=risk_percentage, df = df, i = i, trade_id = id_counter)
            #trade.set_tp(risk)
            trade.position_size = position_size
            print("TEST trade.position_size " +str(trade.position_size))

            #moneyspend = trade.position_size * current_price
            total_position_amount = total_position_amount + position_size
            print("total_position_amount " +str(total_position_amount))
            df.loc[i, "total_position_amount"] = total_position_amount

            trades.append(trade)
            id_counter += 1
        else: 
          print(" ''''''''''''''''''''''''''''''''''''' not enough cash to trade")
        
    # if any trade has hit SL or PT
    
    for trade in trades:

        if trade.status == "open":
            #print("found open trade")
            # 
            
            df.loc[i, "Tp"] = trade.tp1
            if current_price >= trade.tp1 or current_price <= trade.sl:
                #update cash
                trade.status = "closed"
                trade.exit_price = current_price
                print("\n \n EXIT TRADE with id:" + str(trade.trade_id) )
                #update cash
                cash = cash + (trade.position_size* current_price)

                #upate df_trades: 
                
                profit = trade.position_size* current_price - trade.position_size* trade.entry_price
                df_trades.loc[trade.trade_id, "time"] = i

                df_trades.loc[trade.trade_id, "profit"] = profit
                df_trades.loc[trade.trade_id, "max_loss"] = equity * risk_percentage
                df_trades.loc[trade.trade_id, "equity"] = equity
                df_trades.loc[trade.trade_id, "entry_price"] = trade.entry_price               
                df_trades.loc[trade.trade_id, "exit_price"] = current_price
                df_trades.loc[trade.trade_id, "position_size"] = trade.position_size
                df_trades.loc[trade.trade_id, "position*price"] = trade.position_size* trade.entry_price             
                df_trades.loc[trade.trade_id, "cash after sell"] =cash
                df_trades.loc[trade.trade_id, "tp"] = trade.tp1
                df_trades.loc[trade.trade_id, "sl"] = trade.sl


                total_position_amount = total_position_amount - trade.position_size

                if current_price >= trade.tp1:
                  print("winner")
                  df_trades.loc[trade.trade_id, "type"] = "tp"
                  #count winners
                  winning_trades += 1
                  current_winning_streak += 1
                  current_loosing_streak = 0
                  if current_winning_streak > max_winning_streak:
                    max_winning_streak = current_winning_streak

                elif current_price <= trade.sl:
                  print("loss")
                  df_trades.loc[trade.trade_id, "type"] = "sl"
                  
                  #count loosers
                  losing_trades += 1
                  current_losing_streak += 1
                  current_winning_streak = 0

                  if current_losing_streak > max_losing_streak:
                    max_losing_streak = current_losing_streak



print("Total trades: " + str(winning_trades + losing_trades))
print("Winning trades: " + str(winning_trades))
print("Losing trades: " + str(losing_trades))
print("Maximum winning streak: " + str(max_winning_streak))
print("Maximum losing streak: " + str(max_losing_streak))

print("winrate: " + str(100 * winning_trades/(winning_trades + losing_trades)) ) 


backtest


==================================>buy signal <==================================



equity : 10000.0
cash : 10000
total_position_amount before potential trade : 0
current_price: 389.79998779296875
index = 2023-01-19 14:00:00
calculate_position_size
max position size 19
use max position size 19
we can trade
potential_loss 48.25958251953125
potential_spending 7406.199768066406
buy position size 19
cash left 2593.8002319335938
Open Trade ! <-------------
self.trade_id : 0
TEST trade.position_size 19
total_position_amount 19

 
 EXIT TRADE with id:0
winner
==================================>buy signal <==================================



equity : 10077.139953613281
cash : 10077.139953613281
total_position_amount before potential trade : 0
current_price: 399.0
index = 2023-01-25 14:00:00
calculate_position_size
max position size 9
use max position size 9
we can trade
potential_loss 48.96002197265625
potential_spending 3591.0
buy position size 9
cash left 6486.139953613281
Open

In [49]:
df_trades


,time,profit,max_loss,equity,entry_price,exit_price,position_size,position*price,cash after sell,tp,sl,type
trade_id,,,,,,,,,,,,
0,2023-01-20 14:30:00,77.139954,50.385700,10077.139954,389.799988,393.859985,19.0,7406.199768,10077.139954,393.609955,387.260010,tp
1,2023-01-27 15:00:00,76.139923,50.766399,10153.279877,399.000000,407.459991,9.0,3591.000000,10153.279877,407.160004,393.559998,tp
2,2023-02-01 15:00:00,102.509979,51.278949,10255.789856,403.149994,409.179993,17.0,6853.549896,10255.789856,407.454987,400.279999,tp
3,2023-02-07 13:00:00,112.500000,51.841449,10368.289856,409.829987,414.329987,25.0,10245.749664,10368.289856,412.424957,408.100006,tp
4,2023-02-13 09:30:00,56.999969,52.126449,10425.289825,406.440002,408.720001,25.0,10161.000061,10425.289825,408.584991,405.010010,tp


In [50]:
import plotly.graph_objs as go

# Create a scatter plot with x and y data
#mode = "markers"
fig = go.Figure(
    data=go.Scatter(x=df_trades.index, y=df_trades.equity )
    )
fig.update_layout({
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'paper_bgcolor': 'rgba(0,0,0,0)'

})
fig.update_xaxes(tickfont=dict(size=20, family='Arial', color='blue'))

# Update the y-axis tick label font size and style
fig.update_yaxes(tickfont=dict(size=20, family='Arial', color='blue'))
# Set plot title and axis labels
fig.show()

In [51]:
df


,Row_Number,Open,High,Low,Close,Adj Close,Volume,EMA200,EMA500,EMA2000,...,Upper_band,Stop_Loss,Long_Entry,Long_Exit,Potential_loss_over_equity_pct,Cash,Equity,Total_position_amount,total_position_amount,Tp
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-11-28 09:30:00,1,399.089996,400.809998,399.000000,399.867096,399.867096,6205462,399.867096,399.867096,399.867096,...,NaN,NaN,0,0,0.0,10000.000000,10000.000000,0.0,NaN,NaN
2022-11-28 10:00:00,2,399.850006,400.239990,398.825195,399.350006,399.350006,4409803,399.861951,399.865032,399.866579,...,NaN,NaN,0,0,0.0,10000.000000,10000.000000,0.0,NaN,NaN
2022-11-28 10:30:00,3,399.299988,399.549988,398.410004,398.510010,398.510010,3606058,399.848499,399.859622,399.865223,...,NaN,NaN,0,0,0.0,10000.000000,10000.000000,0.0,NaN,NaN
2022-11-28 11:00:00,4,398.510010,398.869995,398.190002,398.540009,398.540009,2539879,399.835479,399.854355,399.863899,...,NaN,NaN,0,0,0.0,10000.000000,10000.000000,0.0,NaN,NaN
2022-11-28 11:30:00,5,398.540009,399.079987,398.350098,398.872009,398.872009,2779621,399.825892,399.850433,399.862907,...,NaN,NaN,0,0,0.0,10000.000000,10000.000000,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-22 13:30:00,760,400.785004,400.880005,400.000000,400.359894,400.359894,2490140,406.366617,402.205585,398.923490,...,402.559998,397.621002,0,0,0.0,10425.289825,10425.289825,0.0,NaN,NaN
2023-02-22 14:00:00,761,400.356293,401.130005,398.600006,399.190002,399.190002,9431314,406.295208,402.193547,398.923756,...,401.720001,397.621002,0,0,0.0,10425.289825,10425.289825,0.0,NaN,NaN
2023-02-22 14:30:00,762,399.190002,399.289703,397.510010,397.589905,397.589905,7549266,406.208588,402.175169,398.922423,...,401.130005,397.510010,0,0,0.0,10425.289825,10425.289825,0.0,NaN,NaN


In [52]:
fig = make_subplots(rows=6, cols=1, vertical_spacing=0.065, shared_xaxes=True, row_heights=[0.4, 0.1, 0.1, 0.1, 0.1, 0.1])


# donchian channel
fig.add_trace(go.Scatter(x=list(df.index), y=list(df.Stop_Loss),line=dict(color='red', width=2), name='lower band'), 1, 1)

#fig.add_trace(go.Scatter(x=list(df.index), y=list(df['Upper_band']),line=dict(color='purple', width=2), name='upper band'), 1, 1)

#add buy signals: 

#sell stoploss reached
jitter = 0.25  
y = df_trades['exit_price']
y_jitter = y + np.random.normal(0, jitter, size=len(y))

fig.add_trace(
    go.Scatter(x=df_trades['time'], y=y_jitter, mode='markers',
                  marker=dict(size=30, color='red', symbol='triangle-down')),1,1)

# buy signal
fig.add_trace(
    go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers',
                  marker=dict(size=30, color='green', symbol='triangle-up')),1,1)

# #add pt1
# df['Tp'] = 0
# fig.add_trace(
#     go.Scatter(x=df.index, y=df['Tp'], mode='markers',
#                   marker=dict(size=30, color='green', symbol='line-ew')),1,1)
                 



# fig.add_trace(go.Candlestick(x=df.index,
#                 open=df['Open'],
#                 high=df['High'],
#                 low=df['Low'],
#                 close=df['Close'],
#                 name='candlestick'),1,1)

fig.add_trace(go.Scatter(x=list(df.index), y=list(df.Close),line=dict(color='blue', width=2), name='Close'), 1, 1)


fig.add_trace(go.Scatter(x=list(df.index), y=list(df.EMA200),line=dict(color='red', width=2), name='EMA200' ), 1, 1)
fig.add_trace(go.Scatter(x=list(df.index), y=list(df.EMA500),line=dict(color='green', width=2), name='EMA500' ), 1, 1)

fig.add_trace(go.Scatter(x=list(df.index), y=list(df.EMA2000),line=dict(color='brown', width=2), name='EMA2000' ), 1, 1)

#second plot
fig.add_trace(go.Scatter(x=list(df.index), y=list(df.MACD),line=dict(color='orange', width=2), name='MACD'), 2, 1);
fig.add_trace(go.Scatter(x=list(df.index), y=list(df.Signal),line=dict(color='blue', width=2), name='Signal'), 2, 1);

# cash
fig.add_trace(go.Scatter(x=list(df.index), y=list(df.Cash),line=dict(color='blue', width=2), name='Cash'), 3, 1);

fig.update_yaxes(
    title_text="Cash",
    title_font=dict(size=25, family="Arial", color="blue"),
    row=3, col=1
)

# equity
fig.add_trace(go.Scatter(x=list(df.index), y=list(df.Equity),line=dict(color='blue', width=2), name='Equity'), 4, 1);

fig.update_yaxes(
    title_text="Equity",
    title_font=dict(size=25, family="Arial", color="blue"),
    row=4, col=1
)

# positions
fig.add_trace(go.Scatter(x=list(df.index), y=list(df.Total_position_amount),line=dict(color='blue', width=2), name='Total_position_amount'), 5, 1);

fig.update_yaxes(
    title_text="Total_position",
    title_font=dict(size=25, family="Arial", color="blue"),
    row=5, col=1
)


# # potential loss

fig.add_trace(go.Scatter(x=list(df.index), y=list(df.Potential_loss_over_equity_pct),line=dict(color='blue', width=2), name='Total_position_amount'), 6, 1);

fig.update_yaxes(
    title_text="potential loss %",
    title_font=dict(size=25, family="Arial", color="blue"),
    row=6, col=1
)

fig.update_layout(height=1000, width=1200, showlegend=True)


fig.update_layout({
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'paper_bgcolor': 'rgba(0,0,0,0)'
})

fig.update_layout(
    legend=dict(
        font=dict(
            family="sans-serif",
            size=18,
            color="blue"
        )
    )
)

fig.update_layout(
    xaxis=dict(
        type="date"),
    #xaxis2_rangeslider_visible=True,
    xaxis2_type="date"
    );
fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]),
        dict(bounds=[16, 9.50], pattern="hour"), #hide hours outside of 9am-5pm
    ]
)
fig.update_xaxes(tickfont=dict(size=20, family='Arial', color='blue'))

# Update the y-axis tick label font size and style
fig.update_yaxes(tickfont=dict(size=20, family='Arial', color='blue'))

fig.show()


#experiment code

In [ ]:
import plotly.graph_objects as go

# your data
x = [1, 2, 3, 4, 5]
y = [2, 4, 6, 8, 10]

# create a horizontal line at 1% of the maximum value
one_percent = 0.01 * max(y)
one_percent_line = go.Scatter(x=[min(x), max(x)], y=[one_percent, one_percent], 
                              mode='lines', line=dict(color='red', dash='dash'))

# create the plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines'))
fig.add_trace(one_percent_line)
fig.show()


In [ ]:
import plotly.graph_objs as go
import numpy as np

# Create sample data
x = [1, 2, 3, 4, 5]
y = [1, 3, 2, 4, 3]
ids = ['A', 'B', 'C', 'D', 'E']

# Add annotations to scatter plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', marker=dict(size=10)))

for i in range(len(x)):
    fig.add_annotation(x=x[i], y=y[i], text=ids[i], font=dict(color='blue', size=40), showarrow=False)

fig.show()


In [ ]:
import plotly.graph_objs as go
import numpy as np

# Create sample data
x = [1,1,2,2]
y = [1,1,2,2]

# Add jitter to y-axis
jitter = 0.1  
y_jitter = y + np.random.normal(0, jitter, size=len(y))


jitter = 0.1  
x_jitter = x + np.random.normal(0, jitter, size=len(x))

# Create scatter plot with jittered data
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_jitter, y=y_jitter, mode='markers', marker=dict(size=5)))

# Show plot
fig.show()


In [ ]:
#buy sugnals: 
df['Long_Entry'] = np.where(
  #macd cross over under 0 line
  (df['MACD'] < 0) & 
  (df['Signal'] < 0) & 
  (df['MACD'].shift(1) < df['Signal'].shift(1)) & 
  (df['MACD'] > df['Signal']) &
  #close above ema200
  (df['Close'] > df['EMA200']) &
  (df['Row_Number'] > window),
   1, 0) 

In [ ]:
import math

class Trade:
    def __init__(self, entry_price, equity, risk_percentage, tp=None, sl=None):
        self.entry_price = entry_price
        self.equity = equity
        self.risk_percentage = risk_percentage
        self.tp = tp
        self.sl = sl
        self.exit_price = None
        self.status = "open"

        self.calculate_position_size()
        self.money_spend_from_capital = self.position_size * self.entry_price

    def calculate_position_size(self):
        risk = self.entry_price - self.sl if self.sl else 0
        one_percent_of_equity = self.equity * self.risk_percentage
        max_position_size = one_percent_of_equity / risk
        self.position_size = min(math.floor(max_position_size), self.equity // self.entry_price)

# Create a trade object
trade = Trade(entry_price=100, equity=1000000, risk_percentage=0.01, sl=90)

# Print the trade object's attributes
print("Entry price:", trade.entry_price)
print("Equity:", trade.equity)
print("Risk percentage:", trade.risk_percentage)
print("Stop loss:", trade.sl)
print("Position size:", trade.position_size)
print("Money spent from capital:", trade.money_spend_from_capital)

moneylost = trade.position_size * risk 
print("Money lost " + str(moneylost))  

Entry price: 100
Equity: 1000000
Risk percentage: 0.01
Stop loss: 90
Position size: 1000
Money spent from capital: 100000


NameError: ignored

ModuleNotFoundError: ignored

In [ ]:
import yfinance as yf

# Download Apple stock data
symbol = "TSLA"
stock = yf.Ticker(symbol)
#data = stock.history(interval="1wk")
df_52= yf.download(tickers="AAPL", period="20y", interval='1wk')
df_52
df_52['DC_upper_52_wk'] = df_52["High"].rolling(window=52).max()

df_52['Over_DC_upper_52_wk'] = np.where(df_52['DC_upper_52_wk'] > df_52['Close'], 1, 0)
df_52
import plotly.graph_objs as go

fig = go.Figure()

# add candlestick chart
fig.add_trace(go.Candlestick(x=df_52.index,
                open=df_52['Open'],
                high=df_52['High'],
                low=df_52['Low'],
                close=df_52['Close'],
                name=symbol))

# add 52-week high line
fig.add_trace(go.Scatter(x=df_52.index, y=df_52['DC_upper_52_wk'], 
                         mode='lines', name='52-week High'))

# add marker for when the stock price is above the 52-week high
for date in df_52[df_52['Over_DC_upper_52_wk'] == 1].index:
    fig.add_trace(go.Scatter(x=[date], y=[df_52.loc[date, 'High']],
                             mode='lines', name='Above 52-week High'))

# update x-axis and y-axis
fig.update_layout(xaxis_rangeslider_visible=False,
                  yaxis_title='Price')

fig.show()


ModuleNotFoundError: ignored